In [1]:
import os
import sys

import logging
import re
import pathlib

import datetime

import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt

from pyplink import PyPlink

import statsmodels.api as sm

from basic_tools import *

"""

jupyter nbconvert 5_1_association.ipynb --to script

python 5_1_association.py height 1 0,1,2



for i in {00..10};do python 5_1_association.py $i 1 0,1,2;done
for i in {11..20};do python 5_1_association.py $i 1 0,1,2;done

for i in {21..30};do python 5_1_association.py $i 1 0,1,2;done
for i in {31..40};do python 5_1_association.py $i 1 0,1,2;done

for i in {41..50};do python 5_1_association.py $i 1 0,1,2;done
for i in {51..60};do python 5_1_association.py $i 1 0,1,2;done

for i in {61..70};do python 5_1_association.py $i 1 0,1,2;done
for i in {71..80};do python 5_1_association.py $i 1 0,1,2;done

for i in {81..90};do python 5_1_association.py $i 1 0,1,2;done
for i in {91..97};do python 5_1_association.py $i 1 0,1,2;done


for i in {00..10};do python 5_1_association.py $i 2 0;done
for i in {11..20};do python 5_1_association.py $i 2 0;done

for i in {21..30};do python 5_1_association.py $i 2 0;done
for i in {31..40};do python 5_1_association.py $i 2 0;done

for i in {41..50};do python 5_1_association.py $i 2 0;done
for i in {51..60};do python 5_1_association.py $i 2 0;done

for i in {61..70};do python 5_1_association.py $i 2 0;done
for i in {71..80};do python 5_1_association.py $i 2 0;done

for i in {81..90};do python 5_1_association.py $i 2 0;done
for i in {91..97};do python 5_1_association.py $i 2 0;done
"""

'\n\njupyter nbconvert 5_1_association.ipynb --to script\n\npython 5_1_association.py height 1 0,1,2\n\n\n\nfor i in {00..10};do python 5_1_association.py $i 1 0,1,2;done\nfor i in {11..20};do python 5_1_association.py $i 1 0,1,2;done\n\nfor i in {21..30};do python 5_1_association.py $i 1 0,1,2;done\nfor i in {31..40};do python 5_1_association.py $i 1 0,1,2;done\n\nfor i in {41..50};do python 5_1_association.py $i 1 0,1,2;done\nfor i in {51..60};do python 5_1_association.py $i 1 0,1,2;done\n\nfor i in {61..70};do python 5_1_association.py $i 1 0,1,2;done\nfor i in {71..80};do python 5_1_association.py $i 1 0,1,2;done\n\nfor i in {81..90};do python 5_1_association.py $i 1 0,1,2;done\nfor i in {91..97};do python 5_1_association.py $i 1 0,1,2;done\n\n\nfor i in {00..10};do python 5_1_association.py $i 2 0;done\nfor i in {11..20};do python 5_1_association.py $i 2 0;done\n\nfor i in {21..30};do python 5_1_association.py $i 2 0;done\nfor i in {31..40};do python 5_1_association.py $i 2 0;done

In [2]:
plink_KCHIP_HLA_AA_SNP_1000G=PyPlink(plink_KCHIP_HLA_AA_SNP_1000G_path)
plink_KCHIP_HLA_AA_SNP_1000G_fam=plink_KCHIP_HLA_AA_SNP_1000G.get_fam().astype({'fid':str,'iid':str}).rename(columns={'fid':'FID','iid':'IID'})
plink_KCHIP_HLA_AA_SNP_1000G_bim=plink_KCHIP_HLA_AA_SNP_1000G.get_bim()

In [8]:
plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_B')]#@[plink_KCHIP_HLA_AA_SNP_1000G_bim]

,chrom,pos,cm,a1,a2
snp,,,,,
HLA_B*07:02,6,31321649,0,p,a
HLA_B*13:01,6,31321649,0,p,a
HLA_B*13:02,6,31321649,0,p,a
HLA_B*14:01,6,31321649,0,p,a
HLA_B*15:01,6,31321649,0,p,a
HLA_B*15:07,6,31321649,0,p,a
HLA_B*15:11,6,31321649,0,p,a
HLA_B*15:18,6,31321649,0,p,a
HLA_B*27:05,6,31321649,0,p,a


In [38]:
plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA') & (~plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains(':'))]

,chrom,pos,cm,a1,a2
snp,,,,,


In [40]:
plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('HLA_A')]

,chrom,pos,cm,a1,a2
snp,,,,,
HLA_A*01:01,6,29910247,0,p,a
HLA_A*02:01,6,29910247,0,p,a
HLA_A*02:06,6,29910247,0,p,a
HLA_A*02:07,6,29910247,0,p,a
HLA_A*03:01,6,29910247,0,p,a
HLA_A*11:01,6,29910247,0,p,a
HLA_A*24:02,6,29910247,0,p,a
HLA_A*26:01,6,29910247,0,p,a
HLA_A*26:02,6,29910247,0,p,a


In [34]:
plink_KCHIP_HLA_AA_SNP_1000G_bim[~plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains(':')]

,chrom,pos,cm,a1,a2
snp,,,,,
AX-11643368,6,29002062,0,G,A
AX-41942515,6,29004091,0,T,C
AX-11688629,6,29006250,0,T,C
AX-11642535,6,29006844,0,G,A
AX-15351471,6,29007847,0,G,C
AX-11594371,6,29009568,0,G,A
AX-11495485,6,29010003,0,C,T
AX-11594778,6,29010153,0,T,G
AX-50494878,6,29010587,0,G,A


In [11]:
plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker('AA_B_325_31322303_exon7_C')

array([0, 1, 1, ..., 1, 0, 0], dtype=int8)

In [10]:
dosage_array=[]
for i in plink_KCHIP_HLA_AA_SNP_1000G_bim[plink_KCHIP_HLA_AA_SNP_1000G_bim.index.str.contains('B_')].index:
    dosage_array.append(plink_KCHIP_HLA_AA_SNP_1000G.get_geno_marker('AA_B_325_31322303_exon7_C'))

AA_B_325_31322303_exon7_C
AA_B_325_31322303_exon7_S
SNPS_B_2633_31322303_exon7
SNPS_B_2620_31322316_intron6
SNPS_B_2596_31322340_intron6_A
SNPS_B_2596_31322340_intron6_G
SNPS_B_2569_31322367_intron6
SNPS_B_2541_31322395_intron6
SNPS_B_2534_31322402_intron6
SNPS_B_2483_31322453_intron5
SNPS_B_2477_31322459_intron5
SNPS_B_2466_31322470_intron5
SNPS_B_2449_31322487_intron5
SNPS_B_2423_31322513_intron5
SNPS_B_2414_31322522_intron5
SNPS_B_2407_31322529_intron5
SNPS_B_2377_31322559_intron5
SNPS_B_2274_31322662_intron5
SNPS_B_2248_31322688_intron5
SNPS_B_2242_31322694_intron5
SNPS_B_2233_31322703_intron5
SNPS_B_2232_31322704_intron5
SNPS_B_2230_31322706_intron5
SNPS_B_2194_31322742_intron5
SNPS_B_2174_31322762_intron5
SNPS_B_2169_31322767_intron5
SNPS_B_2154_31322782_intron5
SNPS_B_2133_31322803_intron5
SNPS_B_2081_31322855_intron5
SNPS_B_2048_31322888_exon5
AA_B_305_31322910_exon5_A
AA_B_305_31322910_exon5_T
SNPS_B_2025_31322911_exon5
AA_B_282_31322979_exon5_I
AA_B_282_31322979_exon5_V
SNPS_